In [1]:
# Tutorial I am Following: https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import keras
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
!mkdir keras_model

mkdir: cannot create directory ‘keras_model’: File exists


In [4]:
!mkdir keras_model

mkdir: cannot create directory ‘keras_model’: File exists


In [5]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'patients_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [6]:
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/patients_model_weights.h5')

In [8]:
# Export the Keras model to the TensorFlow ProtoBuf format

from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

model_version = '1'
export_dir = 'export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [9]:
!ls export

Servo


In [10]:
!ls export/Servo

1


In [11]:
!ls export/Servo/1

saved_model.pb	variables


In [12]:
!ls export/Servo/1/variables

variables.data-00000-of-00001  variables.index


In [13]:
#Tar the entire directory and upload to Amazon S3

import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')


In [14]:
# Deploy 

!touch train.py
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [15]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

-----------!CPU times: user 434 ms, sys: 43.2 ms, total: 478 ms
Wall time: 5min 32s


In [16]:
endpoint_name = 'sagemaker-tensorflow-2020-04-26-10-42-56-305'

In [17]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

In [72]:
# testing the API

import numpy as np

# Input of shape [Age, Respiratory, Pulse]
data = np.matrix([20, 180, 80])
predictor.predict(data)

{'outputs': {'score': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 1}]},
   'float_val': [0.18431062996387482]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1},
  'signature_name': 'serving_default'}}

In [73]:
# 'float_val': 0 - 0.2 very critical , 0.2 - 0.4 critical, 0.4 < normal

In [74]:
# import json
# json.dumps(data.tolist())

In [76]:
import json
import boto3
import numpy as np
import io
 
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
# data = np.random.randn(1, 50).tolist()
response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data.tolist()))
response_body = response['Body']
print(response_body.read())

b'{\n  "outputs": {\n    "score": {\n      "dtype": "DT_FLOAT", \n      "floatVal": [\n        0.18431062996387482\n      ], \n      "tensorShape": {\n        "dim": [\n          {\n            "size": "1"\n          }, \n          {\n            "size": "1"\n          }\n        ]\n      }\n    }\n  }, \n  "modelSpec": {\n    "version": "1", \n    "name": "generic_model", \n    "signatureName": "serving_default"\n  }\n}'


In [80]:
json.dumps(data.tolist())

'[[20, 180, 80]]'